**Script Gene 1258685**

Grupo 5:\
José Campos
PG54900

**Configuração Inicial do meu script**

In [2]:
pip install Biopython

Note: you may need to restart the kernel to use updated packages.


In [3]:
from Bio import Entrez, SeqIO
import os

In [4]:
Entrez.email = "josedrdcampos@gmail.com" #Utilização do meu e-mail porque o NCBI bloqueia requisições sem email

if not os.path.exists("NCBI_downloads"): #Criação de uma pasta para guardar os ficheiros
  os.mkdir("NCBI_downloads")


**Função para procurar o gene através do gene ID**

In [5]:
def procuragenes(gene_id="1258685"): #Função para procurar o geneID na base de dados do NCBI
  handle = Entrez.esearch(db="gene",term="1258685", retmax=10) #retmax será o número de pesquisas máximas que o NCBI vai devolver
  record = Entrez.read(handle)
  print(record)
  return record["IdList"]

**Função para obter registo do Gene no GenBank**

In [6]:
def buscagene(gene_id):
  handle_search = Entrez.esearch(db="nucleotide", term=f"{gene_id}[Gene ID]", retmax=1)
  record_search = Entrez.read(handle_search)
  nucleotide_ids = record_search["IdList"]

  if not nucleotide_ids:
    print(f"Aviso: Nenhuma sequência nucleotídica encontrada para o gene ID {gene_id}.")
    return None

  seq_id = nucleotide_ids[0]
  print(f"A descarregar sequência nucleotídica principal para o Gene ID {gene_id}, usando nucleotide ID: {seq_id}")

  handle_fetch = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
  data = handle_fetch.read()

  nomeficheiro = f"NCBI_downloads/gene_main_nucleotide_{gene_id}.gb"
  with open(nomeficheiro,"w") as f:
    f.write(data)

  print(f"\nFoi guardado o ficheiro da sequência nucleotídica principal para:{nomeficheiro}")
  return nomeficheiro

**Função para pesquisar variantes na base de dados de nucleótidos**

In [7]:
def procuravariantes(gene_id="1258685"):
  handle = Entrez.esearch(db="nucleotide", term=f"{gene_id}[Gene ID]", retmax = 50)
  record = Entrez.read(handle)
  print(record["IdList"])
  return record["IdList"]

**Função para descarregar cada variante no Genbank**

In [8]:
def procuraregistos(id_list):
  gb_files=[]

  for sid in id_list:
    handle = Entrez.efetch(db="nucleotide", id = sid, rettype="gb", retmode = "text")
    text = handle.read()

    nomeficheiro = f"NCBI_downloads/{sid}.gb"
    with open(nomeficheiro, "w") as f:
      f.write(text)

    gb_files.append(nomeficheiro)
    print(f"Guardado: {nomeficheiro}")

  return gb_files

**Função para ler e analisar as features dentro do GenBank**

In [9]:
def analisafeatures(genbank_file):
  record = SeqIO.read(genbank_file,"genbank")

  for feature in record.features:
    print(f"{feature.type}")
    for key, value in feature.qualifiers.items():
      print(f"  - {key}: {value}")

**Função para extrair as features do CDS do gene que codifica a DNA polimerase (T4p047)**

In [10]:
def dna_pol_info(genbank_file):
  record = SeqIO.read(genbank_file,"genbank")

  for feature in record.features:
    if feature.type=="CDS":
      nomegene = feature.qualifiers.get("gene",["?"])[0]
      produto = feature.qualifiers.get("product",[""])[0]

      if nomegene in ["T4p047"] or "dna polymerase" in produto.lower():
        print("\n>>> CDS correspondente à DNA polymerase (T4p047) encontrado!")
        print("   Gene:", nomegene)
        print("   Product:", produto)
        print("   Protein ID:", feature.qualifiers.get("protein_id"))
        print("   db_xref:", feature.qualifiers.get("db_xref"))
        print("   notes:", feature.qualifiers.get("note"))
        print("   sequence length (aa):",
            len(feature.qualifiers.get("translation", [""])[0]))

        seq_aa = feature.qualifiers.get("translation",[""])[0]
        ##para guardar a sequência##
        with open("NCBI_downloads/dna_pol_protein.faa","w") as f:
          f.write(f">dna_pol_protein\n{seq_aa}\n")

          print("\nSequência proteica guardada em dna_pol_protein.faa\n")

**Correr o Script**

In [11]:
def main():
  #1) procurar o gene
  ids = procuragenes("1258685")
  if not ids:
    print("Erro: Nenhum gene encontrado para o ID fornecido.")
    return

  gene_ncbi_id = ids[0]

  #2) descarregar o registo principal do gene (agora um registo nucleotídico)
  ficheiro_gene = buscagene(gene_ncbi_id)
  if ficheiro_gene is None:
      print("Erro: Não foi possível descarregar a sequência nucleotídica principal para o gene.")
      return

  #3) procurar os variantes nucleotídicas
  variant_ids = procuravariantes("1258685")
  if not variant_ids:
    print("Aviso: Nenhuma variante encontrada para o gene ID fornecido.")

  #4) Fazer download das variantes
  if variant_ids:
    ficheiros_variantes = procuraregistos(variant_ids)
  else:
    ficheiros_variantes = []

  #5) Analise do registo principal (agora é um registo nucleotídico válido)
  analisafeatures(ficheiro_gene)

  #6) Extrair informação específica do CDS gp5
  dna_pol_info(ficheiro_gene)

  print("\n=== ANÁLISE TERMINADA ===")


#Vamos correr o script!

if __name__ == "__main__":
  main()


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['1258685'], 'TranslationSet': [], 'TranslationStack': [{'Term': '1258685[UID]', 'Field': 'UID', 'Count': '-1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': '1258685[UID]'}
A descarregar sequência nucleotídica principal para o Gene ID 1258685, usando nucleotide ID: 29366675

Foi guardado o ficheiro da sequência nucleotídica principal para:NCBI_downloads/gene_main_nucleotide_1258685.gb
['29366675']
Guardado: NCBI_downloads/29366675.gb
source
  - organism: ['Escherichia phage T4']
  - mol_type: ['genomic DNA']
  - host: ['Escherichia coli']
  - db_xref: ['taxon:2681598']
gene
  - gene: ['rIIA']
  - locus_tag: ['T4p001']
  - db_xref: ['GeneID:1258593']
CDS
  - gene: ['rIIA']
  - locus_tag: ['T4p001']
  - note: ['membrane integrity protector; mutants give rapid lysis on various lysogenic strains due to effects of specific prophage products, mistakenly interpreted as related to lysis inhibition']
  - codon_start: ['1']
  - transl_ta

**Vamos agora realizar o BLAST do ficheiro obtido**

Código reaproveitado da Aula

In [22]:
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
##import dos packages

In [24]:
pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [gdown]32m5/9 [charset_normalizer]
Note: you may need to restart the kernel to use updated packages.


In [34]:
import os

urls = [
    "https://drive.google.com/file/d/1mWO9KGsLTtmhpzIaoNRUA0UmCCYnHy1K/view?usp=drive_link" ##link do ficheiro FASTA da proteína DNA polimerase (T4p047)
    ]

for url in urls:
    !gdown {url} --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1mWO9KGsLTtmhpzIaoNRUA0UmCCYnHy1K
To: /home/jose-campos/lb/bioinflabs_containers/Trabalho LB/dna_pol_protein.faa
100%|██████████████████████████████████████████| 246/246 [00:00<00:00, 1.85MB/s]


In [35]:
seq = SeqIO.read("dna_pol_protein.faa", "fasta")
print(len(seq.seq))

228


In [41]:
result_handle = NCBIWWW.qblast("blastp", "nr", seq.format("fasta"))

In [42]:
with open("seqprot-blast.xml", "w") as save_file:
    save_file.write(result_handle.read())

result_handle.close()  # Close the result handle if it's still open


In [43]:
result_handle = open("seqprot-blast.xml")
blast_record = NCBIXML.read(result_handle)

print ("PARAMETERS:")
print ("Database: " + blast_record.database)
print ("Matrix: " + blast_record.matrix)
print ("Gap penalties: ", blast_record.gap_penalties)

PARAMETERS:
Database: nr
Matrix: BLOSUM62
Gap penalties:  (11, 1)


In [44]:
print ("number hits: ", len(blast_record.alignments))
first_alignment = blast_record.alignments[0]

print ("FIRST ALIGNMENT:")
print ("Accession: " + first_alignment.accession)
print ("Hit id: " + first_alignment.hit_id)
print ("Definition: " + first_alignment.hit_def)
print ("Alignment length: ", first_alignment.length)
print ("Number of HSPs: ", len(first_alignment.hsps))

number hits:  50
FIRST ALIGNMENT:
Accession: 6DRT_A
Hit id: pdb|6DRT|A
Definition: Chain A, DNA polymerase clamp [Tequatrovirus T4] >pdb|6DRT|B Chain B, DNA polymerase clamp [Tequatrovirus T4] >pdb|6DRT|C Chain C, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|G Chain G, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|H Chain H, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|I Chain I, DNA polymerase clamp [Tequatrovirus T4]
Alignment length:  236
Number of HSPs:  1


In [45]:
hsp = first_alignment.hsps[0]
print ("E-value: ", hsp.expect)
print ("Score: ", hsp.score)
print ("Length: ", hsp.align_length)

print ("Identities: ", hsp.identities)

print (hsp.query[:90])
print (hsp.match[:90])
print (hsp.sbjct[:90])

print (hsp.query[90:])
print (hsp.match[90:])
print (hsp.sbjct[90:])

E-value:  5.73511e-165
Score:  1197.0
Length:  228
Identities:  228
MKLSKDTTALLKNFATINSGIMLKSGQFIMTRAVNGTTYAEANISDVIDFDVAIYDLNGFLGILSLVNDDAEISQSEDGNIKIADARSTI
MKLSKDTTALLKNFATINSGIMLKSGQFIMTRAVNGTTYAEANISDVIDFDVAIYDLNGFLGILSLVNDDAEISQSEDGNIKIADARSTI
MKLSKDTTALLKNFATINSGIMLKSGQFIMTRAVNGTTYAEANISDVIDFDVAIYDLNGFLGILSLVNDDAEISQSEDGNIKIADARSTI
FWPAADPSTVVAPNKPIPFPVASAVTEIKAEDLQQLLRVSRGLQIDTIAITVKEGKIVINGFNKVEDSALTRVKYSLTLGDYDGENTFNFIINMANMKMQPGNYKLLLWAKGKQGAAKFEGEHANYVVALEADSTHDF
FWPAADPSTVVAPNKPIPFPVASAVTEIKAEDLQQLLRVSRGLQIDTIAITVKEGKIVINGFNKVEDSALTRVKYSLTLGDYDGENTFNFIINMANMKMQPGNYKLLLWAKGKQGAAKFEGEHANYVVALEADSTHDF
FWPAADPSTVVAPNKPIPFPVASAVTEIKAEDLQQLLRVSRGLQIDTIAITVKEGKIVINGFNKVEDSALTRVKYSLTLGDYDGENTFNFIINMANMKMQPGNYKLLLWAKGKQGAAKFEGEHANYVVALEADSTHDF


In [46]:
for i in range(5):
    alignment = blast_record.alignments[i]
    print ("Accession: " + alignment.accession)
    print ("Definition: " + alignment.hit_def)
    for hsp in alignment.hsps:
        print ("E-value: ", hsp.expect)

Accession: 6DRT_A
Definition: Chain A, DNA polymerase clamp [Tequatrovirus T4] >pdb|6DRT|B Chain B, DNA polymerase clamp [Tequatrovirus T4] >pdb|6DRT|C Chain C, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|G Chain G, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|H Chain H, DNA polymerase clamp [Tequatrovirus T4] >pdb|7D7D|I Chain I, DNA polymerase clamp [Tequatrovirus T4]
E-value:  5.73511e-165
Accession: WP_015969212
Definition: MULTISPECIES: LCP family protein [Bacteria] >ref|NP_049666.1| DNA polymerase processivity factor [Escherichia phage T4] >sp|P04525.3| RecName: Full=Sliding clamp; AltName: Full=DNA polymerase accessory protein Gp45; AltName: Full=DNA polymerase clamp; AltName: Full=Gene product 45; Short=gp45; AltName: Full=Sliding clamp Gp45 [Tequatrovirus T4] >pdb|1CZD|A Chain A, DNA POLYMERASE ACCESSORY PROTEIN G45 [Tequatrovirus T4] >pdb|1CZD|B Chain B, DNA POLYMERASE ACCESSORY PROTEIN G45 [Tequatrovirus T4] >pdb|1CZD|C Chain C, DNA POLYMERASE ACCESSORY PROTEIN G4

**Alinhamento Múltiplo**

In [ ]:
This code snippet is importing the `os` module and defining a list `urls` containing a single URL link. It then uses the `gdown` command to download a file from the specified URL. The `!` before `gdown` indicates that this code is likely being run in a Jupyter notebook or a similar environment that supports shell commands within the code cells. The `--fuzzy` flag may be used to perform a fuzzy matching during the download process.
import os

urls = [
    "https://drive.google.com/file/d/1mWO9KGsLTtmhpzIaoNRUA0UmCCYnHy1K/view?usp=drive_link" ##link do fasta da proteina codificada pelo meu gene
    ] ## ficheiro obtido utilizando o seguinte site: https://www.ebi.ac.uk/jdispatcher/msa/clustalo

for url in urls:
    !gdown {url} --fuzzy



Downloading...
From: https://drive.google.com/uc?id=1mWO9KGsLTtmhpzIaoNRUA0UmCCYnHy1K
To: /home/jose-campos/lb/bioinflabs_containers/Trabalho LB/dna_pol_protein.faa
100%|██████████████████████████████████████████| 246/246 [00:00<00:00, 1.95MB/s]


In [52]:
from Bio import AlignIO
from Bio import SeqIO

alinhamento = AlignIO.read("clustal25.fasta","fasta")
print(alinhamento)

Alignment with 25 rows and 699 columns
--------------------------------------------...--- VFR13065.1
MEMISNNLNWFVGVVEDRMDPLKLGRVRVRVVGLHPPQRAQGDV...DIG WP_345768145.1
--------------------------------------------...DIG HBC9098468.1
--------------------------------------------...DIG HCS4649654.1
--------------------------------------------...DIG YP_010070530.1
--------------------------------------------...--- 6XC0_A
--------------------------------------------...--- WP_411414910.1
--------------------------------------------...DIG WP_411416213.1
--------------------------------------------...--- WP_411401907.1
--------------------------------------------...--- WP_411429911.1
--------------------------------------------...DIG WP_411425539.1
--------------------------------------------...--- WP_411414880.1
MEMISSSLNWFVGVVEDRMDPLKQGRVRVRVVGLHPAQRAQGDV...--- EPJ2511985.1
MEMISNNLNWFVGVVEDRMDPLKLGRVRVRVVGLHPPQRAQGDV...--- HBU6287408.1
------------------MDPLKLGRVRVRVVGLHPPQRAQGDV...--- WPK180

In [53]:

from collections import Counter

consensus_sequence_list = []
alignment_length = alinhamento.get_alignment_length()

for i in range(alignment_length):
    column = alinhamento[:, i]

    counts = Counter(column)
    most_common_char = counts.most_common(1)[0][0]
    consensus_sequence_list.append(most_common_char)

consensus = "".join(consensus_sequence_list)

print("Consenso:")
print(consensus)

Consenso:
-------------------------------VGLHP-QRAQGDVMGIPT-KLPWMSVIQPITSAAMSGIGGSVTGPVEGTRVYGHFLDKWKTNGIVLGTYGGIVREKPNRLEGFSDPTGQYPRRLGNDTNVLNQGGEVGYDSSSNVIQDSNLDTAINPDDRPLSEIPTDDNPNMSMAEMLRRDEGLRLKVYWDTEGYPTIGIGHLIMKQPVRDMAQINKVLSKQVGREITGNPGSITMEEATTLFERDLADMQRDIKSHSKVGPVWQAVNRSRQMALENMAFQMGVGGVAKFNTMLTAMLAGDWEKAYKAGRDSLWYQQTKGRASRVTMIILTGNLESYGVEVKTPARSLSA--MAA-VAKSSDPADPPIPNDSRILFKEPVSSYKGEYPYVHTMETESGHIQEFD-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


**Árvore Filogenética**

In [51]:
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio import Phylo
import matplotlib.pyplot as plt

calculator = DistanceCalculator()
dist_matrix = calculator.get_distance(alinhamento)

constructor = DistanceTreeConstructor()
tree = constructor.nj(dist_matrix)

Phylo.write(tree, "arvore.nwk", "newick")



Phylo.draw_ascii(tree)




   _ YP_010070529.1
  |
  |, HCS4649669.1
 ,||
 |||      __ WP_411416305.1
 |||     |
 |||     |  , WP_411401073.1
 |||     |  |
 | |     |  |          ___ VFR13065.1
 | |     |  |         |
 | |     |  |         |    ___ WP_442825998.1
 | |     |  |         |   |
 | |_____|  |         | __|         _ WP_411414910.1
 |       |  |         ||  |    ____|
 |       |  |     ____||  |   |    | 6XC0_A
 |       |  |    |    ||  |___|
 |       |  |    |    ||      |  ___ WP_411401907.1
 |       |  |    |    ||      | |
 |       |  |    |    ||      |_|      ____ WP_411429911.1
 |       |  |    |    ||        |     |
 |       |__|    |    ||        |     |              ____ WP_411414880.1
 |          |    |    ||        |_____|        _____|
 |          |    |     |              |       |     |      _ WP_411416213.1
 |          |    |     |              |       |     |_____|
 |          |    |     |              |_______|           |__ HCS4649654.1
_|          |    |     |                      